In [ ]:
# https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class5_class_reg.ipynb

In [ ]:
from matplotlib.pyplot import figure, show
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [ ]:
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

In [ ]:
filename_read = r"c:\ YOUR PATH TO CLEAN DATA \cleaned_data.csv"
df = pd.read_csv(filename_read,na_values=['NA','?'])

In [ ]:
# Encode to a 2D matrix for training
# X: data
# Y: label
x,y = to_xy(df,'Salary')

In [ ]:
# Split data into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [ ]:
# Create NN
# Squential network with 11 layers using a sigmoid
# Optimication is stochastic gradient descent

# https://keras.io
# https://keras.io/layers/core/
# https://keras.io/losses/
# https://keras.io/optimizers/

model = Sequential()
model.add(Dense(1400, input_dim=x.shape[1], activation='sigmoid'))
model.add(Dense(700, activation='sigmoid'))
model.add(Dense(350, activation='sigmoid'))
model.add(Dense(175, activation='sigmoid'))
model.add(Dense(88, activation='sigmoid'))
model.add(Dense(50, activation='sigmoid'))
model.add(Dense(25, activation='sigmoid'))
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(5, activation='sigmoid'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])


In [ ]:
# save best model
#checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=1, save_best_only=True)
#filename_write= r"C:\Users\fr23505\Documents\machine\git\data\best_weights.hdf5"
#checkpointer = ModelCheckpoint(filepath=filename_write, verbose=1, save_best_only=True)

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),verbose=1,epochs=1000)

In [ ]:
# load weights from best model (training above)
#model.load_weights(filename_write)
#model.load_weights('best_weights.hdf5')

In [ ]:
# Predict and measure RMSE
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Score (RMSE): {}".format(score))

In [ ]:
# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

In [ ]:
# Plot the chart
chart_regression(pred.flatten(),y_test)

In [ ]:
chart_regression(pred.flatten(),y_test,sort=False)